In [1]:
# reload magics
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from copy import deepcopy
import datasets
import json

from project_modules.utils import MPutils
from project_modules.io import load_dataset_to_df

## LOAD DATA

In [3]:
from pathlib import Path

# input_path = Path("../lc-project-data")
# output_path = Path("../lc-project-data")

# for MP,
input_path = Path("../Data/DataV3")
output_path = Path("../Data/DataV3")

In [4]:
#NOTE - this is the dataset that has been preprocessed and saved as an arrow file by 01-inspect-data-MP.ipynb
ds = datasets.load_from_disk('../Data/DataV3/2024-04-30-features.arrow')
df = datasets.arrow_dataset.Dataset.to_pandas(ds)

In [5]:
df

,sex,age,SITE,MED-Analgesics,MED-Anesthetics,MED-Antibacterials,MED-Anticonvulsants,MED-Antidepressants,MED-Antifungals,MED-Antimigraine-agents,...,LABS-IND-triglycerides,LABS-IND-urea,LABS-IND-wbc,SUBST-1,SUBST-2,SUBST-3,SUBST-4,SUBST-5,LC_STATUS,__index_level_0__
0,2.0,30.0,2,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,LC_POS,0
1,1.0,33.0,2,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,LC_POS,1
2,2.0,36.0,2,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,LC_POS,2
3,2.0,50.0,2,0,0,0,0,0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,LC_POS,3
4,1.0,40.0,2,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,LC_POS,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5659,1.0,48.0,3,0,0,0,0,0,0,0,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,LC_POS,5659
5660,2.0,42.0,3,0,0,0,0,0,0,0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,LC_POS,5660
5661,1.0,23.0,3,0,0,0,0,0,0,0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,LC_POS,5661
5662,1.0,71.0,3,0,0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,LC_POS,5662


# Check the LC_Status Value counts

In [6]:
df.LC_STATUS.value_counts()

LC_STATUS
LC_NEG    3367
LC_POS    2215
HC          82
Name: count, dtype: int64

In [7]:
df.shape

(5664, 437)

### Total is 5664, LC_Neg=3367, LC_Pos=2215, HC=82

# Drop the 21 patients

In [8]:
df.shape

(5664, 437)

In [9]:
# THE ORIGINAL DATASET from APril 26 --- used to get match the dataset __index__ to the record ids with the index in the original dataset
dfOrg = pd.read_excel("../Data/Original Data/2024_04_26/FINAL-CORRECTED-DATA-2024-04-26.xlsx")

In [10]:
recRemove = ['353','363','400','1020','1147','1154','1159','1164','1171','1173','1228','1242','1272','1300','1325','1344','1345','1346','1348','1349','1380']
recRemove = [int(x) for x in recRemove]

In [11]:
indexOrg = dfOrg[~dfOrg['record_id'].isin(recRemove)].index #only include those records who's record ids are not in the recRemove List

In [12]:
indexOrg.shape

(5643,)

In [13]:
df = df[df['__index_level_0__'].isin(indexOrg)]

In [14]:
df

,sex,age,SITE,MED-Analgesics,MED-Anesthetics,MED-Antibacterials,MED-Anticonvulsants,MED-Antidepressants,MED-Antifungals,MED-Antimigraine-agents,...,LABS-IND-triglycerides,LABS-IND-urea,LABS-IND-wbc,SUBST-1,SUBST-2,SUBST-3,SUBST-4,SUBST-5,LC_STATUS,__index_level_0__
0,2.0,30.0,2,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,LC_POS,0
1,1.0,33.0,2,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,LC_POS,1
2,2.0,36.0,2,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,LC_POS,2
3,2.0,50.0,2,0,0,0,0,0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,LC_POS,3
4,1.0,40.0,2,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,LC_POS,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5659,1.0,48.0,3,0,0,0,0,0,0,0,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,LC_POS,5659
5660,2.0,42.0,3,0,0,0,0,0,0,0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,LC_POS,5660
5661,1.0,23.0,3,0,0,0,0,0,0,0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,LC_POS,5661
5662,1.0,71.0,3,0,0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,LC_POS,5662


In [15]:
df.LC_STATUS.value_counts()

LC_STATUS
LC_NEG    3367
LC_POS    2194
HC          82
Name: count, dtype: int64

### Total=5643, LC_Neg=3367, LC_Pos=2194, HC=82, Here we match Tanya's data with the LC_Status Numbers

# Inspect Missing Data

In [16]:
# Show rows with missing data
dfMissing = df[df.isnull().any(axis=1)]
dfMissing

,sex,age,SITE,MED-Analgesics,MED-Anesthetics,MED-Antibacterials,MED-Anticonvulsants,MED-Antidepressants,MED-Antifungals,MED-Antimigraine-agents,...,LABS-IND-triglycerides,LABS-IND-urea,LABS-IND-wbc,SUBST-1,SUBST-2,SUBST-3,SUBST-4,SUBST-5,LC_STATUS,__index_level_0__
3333,NaN,NaN,4,0,0,0,0,0,0,0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,LC_POS,3333
3710,NaN,53.0,5,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,LC_NEG,3710
3725,NaN,54.0,5,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,LC_NEG,3725
3827,NaN,76.0,5,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,LC_NEG,3827
3905,NaN,41.0,5,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,LC_NEG,3905
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4888,NaN,NaN,5,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,LC_NEG,4888
4913,NaN,NaN,5,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,LC_NEG,4913
4914,NaN,NaN,5,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,LC_NEG,4914
5426,1.0,NaN,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,LC_NEG,5426


### There are still 66 rows with missing values in the dataset

In [17]:
dfMissing.LC_STATUS.value_counts()

LC_STATUS
LC_NEG    61
LC_POS     5
Name: count, dtype: int64

In [ ]:
df.columns[df.isnull().any()].tolist()

['sex', 'age']

### MISSING DATA: here there are 5 LC_POS patients and 61 LC_NEG patients with missing data in AGE and Sex Columns

# Thus the complete dataset with no missing values is as follows


In [20]:
dfNoMissing = df.dropna()

In [21]:
dfNoMissing.shape

(5577, 437)

In [22]:
dfNoMissing.LC_STATUS.value_counts()

LC_STATUS
LC_NEG    3306
LC_POS    2189
HC          82
Name: count, dtype: int64